In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from tqdm.asyncio import tqdm

from prompts import load_template

In [ ]:
load_dotenv()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
CATEGORY = "category"
ANSWER = "answer"
REASONING = "reasoning"

In [ ]:
# 현재 실험은 1.3.x 입니다.
df = pd.read_csv("data/experiments/train_exp_outer_inference.csv")
df.head()

,id,paragraph,question,choices,answer,question_plus,reasoning,category,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_page,keyword_2_page,keyword_3_page,keyword_4_page,keyword_5_page
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,외적 추론,송준길,서인,예송논쟁,인조반정,탕평 정치,"송준길(宋浚吉, 1606년 12월 28일 ~ 1672년 12월 2일)은 조선시대 후...","서인(西人)은 조선 시대 붕당정치 초기의 당파로, 1575년(선조 8)의 동서분당(...","예송(禮訟, 문화어: 례송) 또는 예송논쟁(禮訟論爭, 문화어: 례송론쟁)은 예절에 ...","인조반정(仁祖反正)은 1623년 4월 11일(음력 3월 12일) 김류, 이귀, 신경...",NaN
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...",외적 추론,삼한,고려사,동녕부,몽골,지눌,"삼한(三韓)은 삼국 시대 이전, 곧 원삼국 시대에 한반도에 자리 잡고 있던 마한(馬...",고려사(高麗史)는 조선 전기에 편찬된 고려 왕조의 역사를 저술한 역사서이며 기전체(...,동녕부(東寧府)는 원나라가 고려 서경을 편입한 후 설치한 통치기관이다.\n\n역사\...,"몽골국(몽골어: Монгол Улс, ᠮᠣᠩᠭᠣᠯᠤᠯᠤᠰ 멍걸 올스 듣기 , 음차:...","지눌(知訥, 1158년~1210년 4월 22일(음력 3월 27일))은 고려 중기 ~..."
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,외적 추론,소정방,김문영,신라,김춘추,백제,"소정방(蘇定方, 592년 ~ 667년)은 당나라의 장군이다. 소열(蘇烈)이 본명이지...","김문영(金文穎, 생몰년 미상)은 7세기 신라의 장군이자 관인이었다. 김문영(金文永)...","신라(新羅)는 기원전 57년부터 기원후 935년까지 고구려, 백제와 함께 고대 한반...","태종 무열왕(太宗武烈王, 603년 ~ 661년 음력 6월)은 신라(新羅)의 제29대...","백제(百濟)는 한국의 고대 국가 중 하나로, 고구려, 신라와 함께 삼국 시대를 구성..."
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,외적 추론,영조,서원,대전회통,비변사,통상 수교,"영조(英祖, 1694년 10월 31일(음력 9월 13일) ~ 1776년 4월 22일...",서원(書院)는 중국에서 발단 한자 문화권의 동아시아 (일본 한국 베트남)에 파급된...,《대전회통》(大典會通)은 1865년(고종 2년)에 조두순(趙斗淳) 등이 왕명에 따라...,비변사(備邊司)는 조선시대 군국기무(軍國機務)를 관장한 문무합의기구(文武合議機構)다...,NaN
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,외적 추론,대무예,고구려,발해,3성 6부,중앙집권,"무왕(武王, ?~737년)은 발해의 제2대 왕이다. 휘는 무예(武藝)이다. 연호는 ...",고구려(高句麗)는 한국의 고대 왕조국가 중 하나이다. 국성은 고씨(高氏)이다. 본래...,발해(渤海) 또는 발해국(渤海國)은 고구려 멸망 이후 고구려 계승을 표방하던 대조영...,삼성육부제(三省六部制)는 과거 중세 중국 왕조의 중앙 정치 제도이다. 이는 주변국에...,"중앙집권(中央集權, 영어: centralisation, centralization)..."


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="document_summarization.txt", template_type="paragraph_generation")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

summarizer = prompt | llm

In [ ]:
def summarize(row: pd.Series, i: int) -> str:
    """
    위키피디아 문서를 요약하는 함수

    Args:
        row (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나
        i {int}: 참고할 키워드

    Returns:
        str: 추출된 5개 키워드
    """

    keywords_list = [row[f"keyword_{idx}"] for idx in range(1, 6)]
    keywords = ", ".join(keywords_list)

    if pd.isna(row[f"keyword_{i}_page"]):
        return ""
    document = row[f"keyword_{i}_page"]

    response = summarizer.invoke(
        {
            "keywords": keywords,
            "document": document,
        },
    )
    return response.content


tqdm.pandas()
sampled_df = df.sample(20, random_state=1004)
for i in range(1, 6):
    sampled_df[f"keyword_{i}_summary"] = sampled_df.progress_apply(lambda row: summarize(row, i), axis=1)

sampled_df.head()

100%|██████████| 20/20 [01:15<00:00,  3.77s/it]


,id,paragraph,question,choices,answer,question_plus,reasoning,category,keyword_1,keyword_2,...,keyword_1_page,keyword_2_page,keyword_3_page,keyword_4_page,keyword_5_page,keyword_1_summary,keyword_2_summary,keyword_3_summary,keyword_4_summary,keyword_5_summary
158,generation-for-nlp-634,"""학문적인 아리스토텔레스가 플라톤으로 대체되는 일은 비잔틴 학자와의 접촉으로 가속화...",본문 정보에 따르면 메디치 가문이 설립한 학교가 가장 선호한 철학은 다음 중 무엇입니까?,"['신고전주의', '스콜라 철학', '르네상스 인본주의', '신플라톤주의']",4,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 메디치 가문이 설립한 피렌체 아카데...,외적 추론,메디치 가문,피렌체 아카데미,...,메디치가(이탈리아어: Medici)는 13세기부터 17세기까지 피렌체에서 강력한 영...,NaN,"플라톤(고대 그리스어: Πλάτων, 플라톤, ""넓은, 어깨 폭이 넓은"";, 영어:...","아리스토텔레스(고대 그리스어: Ἀριστοτέλης, 그리스어: Αριστοτέλη...","신플라톤주의(Neoplatonism)는 3세기 이후, 플로티노스의 《엔네아데스》를 ...",메디치 가문(이탈리아어: Medici)은 13세기부터 17세기까지 피렌체에서 강력한...,,"플라톤(Plato, 기원전 428/427년 ~ 기원전 348/347년)은 고대 그리...","메디치 가문은 이탈리아 피렌체에서 15세기부터 18세기까지 정치적, 경제적, 문화적...",신플라톤주의(Neoplatonism)는 3세기 이후 플로티노스의 《엔네아데스》를 기...
66,generation-for-nlp-516,다음 인용문은 1755년 리스본 지진에 대한 볼테르의 반응입니다. 친애하는 내 벗에...,볼테르가 토로한 위의 생각들은 계몽주의 지식인의 어떤 특징들을 가장 잘 표현하고 있...,['상당수가 일가를 이룬 과학자들로서 우주에 대한 인간의 이해를 넓히는 데 일조했다...,4,NaN,문제 분석 \n볼테르의 인용문은 1755년 리스본 지진에 대한 그의 반응을 담고 ...,외적 추론,리스본 지진,볼테르,...,1755년 리스본 지진(영어: 1755 Lisbon earthquake)은 1755...,"볼테르(프랑스어: Voltaire, 프랑스어 발음: ​[vɔltɛːʁ])라는 필명으...","계몽주의(啓蒙主義, 프랑스어: Lumières 뤼미에르[*])란 18세기 하반기에 ...",자연 법칙(自然法則) 또는 물리 법칙(物理法則)은 경험적인 관찰에 기초한 과학적인 ...,NaN,"1755년 리스본 지진은 11월 1일 오전 9시 40분에 발생한 대규모 지진으로, ...","볼테르(Voltaire, 1694~1778)는 프랑스의 대표적인 계몽주의 작가로, ...",계몽주의(啓蒙主義)는 18세기 하반기에 프랑스를 중심으로 유럽 전역에서 유행한 문화...,리스본 지진은 1755년 11월 1일 포르투갈의 수도 리스본에서 발생한 대규모 지진...,
40,generation-for-nlp-479,○항일민족교육의 요람인 서전 서숙을 설립하였다.○만국평화회의가 열린 헤이그에 특사로...,다음 설명에 해당하는 인물에 대한 설명으로 옳은 것은?,"['경학사를 조직하였다.', '독립의 군부를 조직하였다.', '대한인 국민회를 조직...",4,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 항일 민족 교육의 중요성을 강조하며...,외적 추론,안창호,서전 서숙,...,"안창호(安昌浩, 1878년 11월 9일~1938년 3월 10일)는 대한제국의 교육개...",NaN,NaN,대한광복군 정부(大韓光復軍政府)는 1914년 러시아 연해주에서 설립된 한국의 무장 ...,NaN,"안창호(安昌浩, 1878년 11월 9일~1938년 3월 10일)는 대한제국의 교육개...",,,안창호(安昌浩)는 대한광복군 정부(大韓光復軍政府)의 설립과 항일 민족 교육에 중요한...,
563,generation-for-nlp-1322,중세의 이론가 중에는 지하드가 방어 전쟁이라고 주장하는 사람도 일부 있었으나… 대부...,두 번째 구절은 다음 중 어떤 방식을 통해 전통적인 역사적 가정을 약화시키는가?,['십자군 시대 중동 지역의 국가 차원의 적대감이 항상 개인이나 경제 생활에 반영되...,1,NaN,"문제 분석\n주어진 지문은 중세 이슬람과 기독교 간의 관계를 다루고 있으며, 특히 ...",외적 추론,지하드,십자군 시대,...,지하드(아랍어: جهاد)는 아랍어로 고투 혹은 분투를 의미한다. 이슬람교 성전은 ...,NaN,무슬림(아랍어: مسلم Muslim)은 이슬람의 제4 분사형으로 '복종의 행위를 ...,"기독교인(基督敎人), 기독교도(基督敎徒), 기독인(基督人) 또는 그리스도인(영어: ...",NaN,"지하드(جهاد)는 아랍어로 '고투' 또는 '분투'를 의미하며, 이슬람에서의 지하드...",,"무슬림(아랍어: مسلم, Muslim)은 이슬람을 믿는 사람을 지칭하며, '복종의...","지하드(Jihad)는 이슬람에서 신의 뜻을 실현하기 위한 노력이나 투쟁을 의미하며,...",
23,generation-for-nlp-454,왕께서는 “우리나라 사람들은 유교 경전과 중국 역사에 대해서는 자세히 말하는 사람이...,다음 내용의 역사서에 대한 설명으로 옳은 것은?,"['불교를 중심으로 신화와 설화를 정리하였다.', '유교적인 합리주의사관에 따라 기...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 왕이 우리나라 역사에 대한 유감과 함께 역...,외적 추론,유교,역사서,...,유교(儒敎)는 중국 춘추시대(기원전 770~403) 말기에 공자(孔子)가 체계화한 ...,"성경의 역사서는 기독교 구약성경의 일부로, 네비임 중 전기 예언서 전부와 케투빔중 ...","삼국(三國, ""세 나라"")은 맥락에 따라 다음을 가리키는 말이다.\n\n역사\n한국...",NaN,NaN,"유교(儒敎)는 중국 춘추시대 말기에 공자(孔子)에 의해 체계화된 사상으로, 동아시아...","유교는 동아시아에서 오랜 역사를 가진 철학적 및 종교적 전통으로, 인간의 도덕적 삶...","삼국(三國)은 여러 맥락에서 사용되는 용어로, 주로 한국의 삼국 시대(기원전 1세기...",,


In [ ]:
# 현재 실험은 1.3.x 입니다.
sampled_df.to_csv("data/experiments/train_exp_v1.3.5.csv", index=False)